In [57]:
import numpy as np
from mido import MidiFile
from hmmlearn import hmm
import constants

In [58]:
#Displays the content of a typical Midi file (with one track - what we will be focusing on). 

midi = MidiFile('midi/18039.mid', clip=True)
print(midi)


MidiFile(type=0, ticks_per_beat=480, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=500000, time=0),
    MetaMessage('key_signature', key='G', time=0),
    Message('note_on', channel=1, note=66, velocity=64, time=1000),
    Message('note_off', channel=1, note=66, velocity=64, time=125),
    Message('note_on', channel=1, note=68, velocity=64, time=0),
    Message('note_off', channel=1, note=68, velocity=64, time=125),
    Message('note_on', channel=1, note=71, velocity=64, time=0),
    Message('note_off', channel=1, note=71, velocity=64, time=250),
    Message('note_on', channel=1, note=70, velocity=64, time=0),
    Message('note_off', channel=1, note=70, velocity=64, time=500),
    Message('note_on', channel=1, note=71, velocity=64, time=0),
    Message('note_off', channel=1, note=71, velocity=64, time=250),
    Message('note_on', channel=1, note=68, velocity=64, time=0),
    Message('note_off', channel=1, note=68, velocity=64, time=250),
    Message('note_on', channel=1, no

In [59]:
#Now, these 'messages' need to be converted into a 2D Array (hmm cannot support 1D arrays).
#There can only be 1 value in each array of the 2D array
import re
#My idea for this is to use the 'note' value of each message - my rationale is that the duration of notes will likely not
#    significantly affect a piece's tonality - but we can explore this later.

#PRE: midi file is well formed, and is indeed a MidiFile object, and re module is imported
#POST: A 1D array containing the sequence of notes is returned
def extractNotes(midi):
    midi_text = str(midi)
    note_pattern = re.compile('note=(\d{1,3}),')
    
    #return removes unnecessary doubling of notes, caused by 'note_on' and 'note_off' messages
    note_list = note_pattern.findall(midi_text)[1::2]

    note_list_int = [int(i) for i in note_list]
    
    return np.reshape(note_list_int, [-1,1])

#Printing this will show each note statement in the midifile given
print(extractNotes(midi))



[[66]
 [68]
 [71]
 [70]
 [71]
 [68]
 [65]
 [66]
 [68]
 [66]
 [65]
 [66]
 [68]
 [70]
 [68]
 [65]
 [66]
 [65]
 [66]
 [65]
 [63]
 [65]
 [66]
 [65]
 [66]
 [65]
 [66]
 [65]
 [66]
 [65]
 [66]
 [65]
 [63]
 [61]
 [59]
 [58]
 [56]
 [59]
 [58]
 [56]
 [58]
 [59]
 [65]
 [68]
 [70]
 [71]
 [70]
 [71]
 [73]
 [75]
 [77]
 [75]
 [73]
 [75]
 [77]
 [78]
 [77]
 [75]
 [73]
 [75]
 [77]
 [78]
 [80]
 [82]
 [80]
 [78]
 [77]
 [78]]


In [60]:
#Now, the HMM needs to be trained on all of the midi files in the test set for tonal and atonal.

#After reading more about HMM's, I learned that the only way to train on the detection of 
#    both tonal and atonal sequences is to create another HMM, for the 'atonal Markov chain.'

#For each HMM, it needs the raw 1D array of data, and and array of the lengths of each chain (length of track).

#I will import the module 'os' to assist with directory and file management.
import os

#import strings representing directories of test, val, and train files

#PRE: Directory is well-defined and contains at least 1 Midifile.
#POST: Markov Chain is created and returned (raw chain data array, with length array)
def createMarkovChain(directory):
    tracks = []
    track_lengths = []
    
    for filename in os.listdir(directory):
        file = os.path.join(directory, filename)
        if os.path.isfile(file):
            extracted = extractNotes(MidiFile(file))
            tracks.extend(extracted)
            track_lengths.append(len(extracted))
            
    return [tracks, track_lengths]

#To show structure, print the output of the two arrays.
print(createMarkovChain(constants.ATONAL_VAL_DIR))


[[array([68]), array([67]), array([68]), array([66]), array([65]), array([66]), array([67]), array([68]), array([67]), array([70]), array([66]), array([67]), array([68]), array([75]), array([74]), array([73]), array([77]), array([79]), array([81]), array([79]), array([80]), array([81]), array([83]), array([82]), array([84]), array([83]), array([80]), array([82]), array([81]), array([84]), array([82]), array([79]), array([78]), array([76]), array([75]), array([74]), array([75]), array([74]), array([75]), array([76]), array([77]), array([75]), array([74]), array([76]), array([75]), array([73]), array([72]), array([73]), array([74]), array([73]), array([74]), array([75]), array([77]), array([75]), array([74]), array([71]), array([72]), array([75]), array([76]), array([73]), array([71]), array([73]), array([70]), array([68]), array([70]), array([71]), array([74]), array([75]), array([76]), array([77]), array([79]), array([80]), array([83]), array([84]), array([83]), array([84]), array([83]

In [120]:
#Training

#Next, I will need to train the tonal HMM and the atonal HMM.

#I will be experimenting with these parameters.

param = createMarkovChain(constants.TONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]

tonal_hmm = hmm.GaussianHMM(n_components=10, n_iter=200).fit(X, lengths)

param = createMarkovChain(constants.ATONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]

atonal_hmm = hmm.GaussianHMM(n_components=10, n_iter=200).fit(X, lengths)

print("Done")
#Simple enough ...


Done


In [74]:
#Evaluation

#The hmm module comes with its own score() function. 
#   It returns a log-likelihood probability (ranges from -infinity to +infinity).

#For one hmm, this is meaningless - but in comparison to the atonal hmm, we can interpret
#   which is more likely - tonality or atonality.

X = extractNotes(MidiFile('midi/18039.mid', clip=True))
length = len(X)

print(tonal_hmm.score(X, length))
print(atonal_hmm.score(X, length))

#In this case, the tonal model correctly predicted the tonality (higher log_prob, barely)

#So, I will need to create an evaluation method to see how well the models will predict tonality
#   (or lack thereof)

-186.21346554639857
-186.55185253972698


In [150]:
#PRE: Models, arrays, and midifiles are well-defined.
#     Constants (paths to midi files) are correctly defined.
#POST: The tonality on each tonal test midi are evaluated,
#      and returned as scored tonal, scored atonal, and total
#      with average scores of each model
def evalTestDirectory(tonal_model, atonal_model, directory):
    scored_tonal = 0
    scored_atonal = 0
    tonal_scores = []
    atonal_scores = []
    
    for filename in os.listdir(directory):
        file = os.path.join(directory, filename)
        if os.path.isfile(file):
            X = extractNotes(MidiFile(file))
            
            tonal_score = tonal_model.score(X)
            atonal_score = atonal_model.score(X)
            tonal_scores.append(tonal_score)
            atonal_scores.append(atonal_score)

            if(tonal_model.score(X) > atonal_model.score(X)):
                scored_tonal+=1
            else:
                scored_atonal+=1
        
    return ([scored_tonal, scored_atonal, np.average(tonal_scores), np.average(atonal_scores),])


#PRE: Models, arrays, and midifiles are well-defined.
#     Constants (paths to midi files) are correctly defined.
#POST: Percentages of correct predictions between the two models will be displayed.
#      This is accomplished by evaluating each midi file in a directory.
#      Results will be output in readable form

def score(atonal_model, tonal_model):
    
    print("Scoring...")
    
    #These are arrays - [#tonal, #atonal, avgTonalScore, avgAtonalScore, total]
    tonal_test = evalTestDirectory(tonal_hmm, atonal_hmm, constants.TONAL_TEST_DIR)
    atonal_test = evalTestDirectory(tonal_hmm, atonal_hmm, constants.ATONAL_TEST_DIR)
    
    correct = tonal_test[0] + atonal_test[1]
    incorrect = tonal_test[1] + atonal_test[0]
    total = correct + incorrect
    score = correct / total
    
    #print("\nTonality:")
    #print("Correct: " , correct)
    #print("Inorrect: " , incorrect)
    #print("Score:" , score)
    
    return score

    
    

In [121]:
#Executing Evaluation

score(tonal_hmm, atonal_hmm)

Scoring...

Tonality:
Correct:  32
Inorrect:  18
Score: 0.64


In [148]:
# With some tweaking, I have noticed a higher scores with higher number of states.
# I think that the optimum n_components is the number of notes in a track file, so I will average those and try that.

param = createMarkovChain(constants.TONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]
estimated_components = 13 

#This was not successful. Initially I set estimated_components = int(np.average(lengths))
# This proved to be incorrect because some tracks are not that long - and n_components should not exceed
#  the total number of messages in a sequence.

tonal_hmm = hmm.GaussianHMM(n_components=int(estimated_components), n_iter=100).fit(X, lengths)

param = createMarkovChain(constants.ATONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]
estimated_components = 13

atonal_hmm = hmm.GaussianHMM(n_components=int(estimated_components), n_iter=100).fit(X, lengths)

print("Done")

Done


In [149]:
score(tonal_hmm, atonal_hmm)

Scoring...

Tonality:
Correct:  34
Inorrect:  16
Score: 0.68


In [152]:
# This was not necessarily correct. I will attempt to calculate better HP's with the RandomSearchCV() method.
# We can use each of these models below as a baseline for comparing against each other.

# I will also try to vaidate the models prior to fitting.

from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV

# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring



In [157]:
#I am going to attempt to use a custom scorer to use with RandomSearchCV(). 

#Initial models (baseline)

param = createMarkovChain(constants.TONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]

tonal_hmm_old = hmm.GaussianHMM(n_components=10, n_iter=200).fit(X, lengths)

param = createMarkovChain(constants.ATONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]

atonal_hmm_old = hmm.GaussianHMM(n_components=10, n_iter=200).fit(X, lengths)

print("Done training base HMM's ")

#Now I will use RandomSearchCV to attempt to find the best parameters...

#In order to do this, I need to make two 'scorer' objects for each HMM classifier

def scoreTonalFunc(tonal_hmm):
    return score(tonal_hmm, atonal_hmm_old)

def scoreAtonalFunc(atonal_hmm):
    return score(tonal_hmm_old, atonal_hmm)

#param_dict = {
    
#    "n_components" : [10, 12, 13, 15],
#    "covariance_type" : ["sperical", "diag", "full", "tied"],
#    "n_iter" : [10, 50, 100, 200],
#    "algorithm" : ["viterbi", "map"]
    
#}

#scoreTonalModel = make_scorer(scoreTonalFunc)
#scoreAtonalModel = make_scorer(scoreAtonalFunc)

#searchTonal = RandomizedSearchCV(hmm.GaussianHMM(), param_dict, cv=5, scoring=(scoreTonalModel))
#searchAtonal = RandomizedSearchCV(hmm.GaussianHMM(), param_dict, cv=5, scoring=(scoreAtonalModel))


#param = createMarkovChain(constants.TONAL_TRAIN_DIR)
#X = param[0]
#lengths = param[1]

#searchTonal.fit(X, lengths)

#param = createMarkovChain(constants.ATONAL_TRAIN_DIR)
#X = param[0]
#lengths = param[1]

#searchAtonal.fit(X, lengths)


#print("Done")



#This didn't work - it seems that hmmlearn is not directly compatible with RandomSearchCV. 

Done training base HMM's 


In [160]:
param = createMarkovChain(constants.TONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]

tonal_hmm = hmm.GaussianHMM(n_components=13, n_iter=500, covariance_type='spherical').fit(X, lengths)

param = createMarkovChain(constants.ATONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]

atonal_hmm = hmm.GaussianHMM(n_components=13, n_iter=500, covariance_type='spherical').fit(X, lengths)

score(tonal_hmm, atonal_hmm)

Scoring...

Tonality:
Correct:  83
Inorrect:  17
Score: 0.83


0.83

In [161]:
#saving models for future use

#getting a little warmer

import pickle
with open("tonal.83.pkl", "wb") as file: pickle.dump(tonal_hmm, file)
with open("atonal.83.pkl", "wb") as file: pickle.dump(atonal_hmm, file)


In [170]:
param = createMarkovChain(constants.TONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]

tonal_hmm = hmm.MultinomialHMM(n_components=13, n_iter=600, implementation="scaling").fit(X, lengths)

param = createMarkovChain(constants.ATONAL_TRAIN_DIR)
X = param[0]
lengths = param[1]

atonal_hmm = hmm.MultinomialHMM(n_components=13, n_iter=600, implementation="scaling").fit(X, lengths)

score(tonal_hmm, atonal_hmm)

Scoring...

Tonality:
Correct:  91
Inorrect:  9
Score: 0.91


0.91

In [171]:
#Alright, we've got a pair of HMM's that score 91% accurately on the test data.

#I'm going to export these now. 

import pickle
with open("models/tonal.91.pkl", "wb") as file: pickle.dump(tonal_hmm, file)
with open("models/atonal.91.pkl", "wb") as file: pickle.dump(atonal_hmm, file)